In [30]:
import pandas as pd
data = pd.read_csv("complete.csv", on_bad_lines='skip')
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
df=data[data["country"]=="us"].sort_index(level="datetime")
df = df.dropna(subset=['datetime'])
df = df.sort_values(by='datetime').reset_index(drop=True)
df=df.dropna()

C:\Users\mich0\AppData\Local\Temp\ipykernel_16168\2108328890.py:2: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("complete.csv", on_bad_lines='skip')


In [31]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Span
from bokeh.io import output_notebook

# output_notebook()  # Uncomment if using Jupyter Notebook

# === Load & Clean Data ===
data = pd.read_csv("complete.csv", on_bad_lines='skip')
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
df = data[data["country"] == "us"].dropna(subset=['datetime'])
df['year'] = df['datetime'].dt.year
year_counts = df['year'].value_counts().sort_index()

sightings_source = ColumnDataSource(data=dict(
    year=year_counts.index.tolist(),
    count=year_counts.values.tolist()
))

# === Annotated Media Events ===
alien_media = {
    1951: ("The Day the Earth Stood Still", "movie"),
    1977: ("Close Encounters", "movie"),
    1979: ("Alien", "movie"),
    1982: ("E.T.", "movie"),
    1986: ("Aliens", "movie"),
    1993: ("The X-Files (TV)", "tv"),
    1996: ("Independence Day", "movie"),
    1998: ("X-Files: The Movie", "tv"),
    2002: ("Signs", "movie"),
    2009: ("District 9", "movie"),
    2014: ("Edge of Tomorrow", "movie")
}

media_data = {
    'year': [],
    'count': [],
    'title': [],
    'type': []
}
for year, (title, typ) in alien_media.items():
    if year in year_counts.index:
        media_data['year'].append(year)
        media_data['count'].append(year_counts[year])
        media_data['title'].append(title)
        media_data['type'].append(typ)

media_source = ColumnDataSource(media_data)

# === Create Bokeh Plot ===
p = figure(
    width=1000, height=500,
    background_fill_color="#0f0f1f", border_fill_color="#0f0f1f",
    title="U.S. UFO Sightings with Major Alien Media Releases",
    x_axis_label=None, y_axis_label=None,
    x_range=(min(year_counts.index)-1, max(year_counts.index)+1),
    tools="", toolbar_location=None
)

p.vbar(x='year', top='count', width=0.8, source=sightings_source,
       color="#00ff99", line_color=None, legend_label="Sightings")

# Neon markers for media events
media_renderer = p.circle(x='year', y='count', size=10,
                          color="#ff3366", alpha=0.9, source=media_source,
                          legend_label="Media Event")

# Tooltip for media events
p.add_tools(HoverTool(
    renderers=[media_renderer],
    tooltips=[("Media", "@title"), ("Year", "@year"), ("Sightings", "@count")],
    mode='mouse'
))

# Styling polish
p.xaxis.major_label_text_color = "#cccccc"
p.yaxis.major_label_text_color = "#cccccc"
p.xaxis.axis_line_color = "#333333"
p.yaxis.axis_line_color = "#333333"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "#222244"
p.title.text_color = "#eeeeee"
p.title.text_font_size = "16pt"
p.title.align = "center"
p.legend.label_text_color = "#cccccc"
p.legend.background_fill_alpha = 0.0
p.legend.location = "top_left"

# Show it
show(p)


C:\Users\mich0\AppData\Local\Temp\ipykernel_16168\813289658.py:9: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("complete.csv", on_bad_lines='skip')


In [32]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category20
from bokeh.models import Legend
import numpy as np

# Load and clean data
data = pd.read_csv("complete.csv", on_bad_lines='skip')
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
df = data[data["country"] == "us"].dropna(subset=['datetime', 'shape'])
df['year'] = df['datetime'].dt.year

# Group by year and shape
shape_counts = df.groupby(['year', 'shape']).size().unstack(fill_value=0)

# Sort shapes by total frequency and limit to top N
top_shapes = shape_counts.sum().sort_values(ascending=False).head(6).index
shape_counts = shape_counts[top_shapes]

# Prepare stacked data
shape_counts_sorted = shape_counts.sort_index()
source_data = {'year': shape_counts_sorted.index.tolist()}
colors = Category20[20][:len(top_shapes)]
for shape in top_shapes:
    source_data[shape] = shape_counts_sorted[shape].values

source = ColumnDataSource(data=source_data)

# Create locked figure (no pan/zoom)
p = figure(
    width=1000, height=500,
    x_range=(shape_counts_sorted.index.min(), shape_counts_sorted.index.max()),
    y_range=(0, shape_counts_sorted.sum(axis=1).max() * 1.1),
    background_fill_color="#0f0f1f",
    title="U.S. UFO Shape Distribution Over Time",
    tools="",  # disable interactive tools
    toolbar_location=None
)

# Stack areas
renderers = p.varea_stack(stackers=list(top_shapes), x='year',
                          color=colors, legend_label=list(top_shapes), source=source)

# Add hover
hover = HoverTool(
    tooltips=[("Year", "@year")] + [(shape, f"@{shape}") for shape in top_shapes],
    mode='vline'
)
p.add_tools(hover)

# Styling
p.xaxis.axis_label = "Year"
p.yaxis.axis_label = "Sightings"
p.xaxis.major_label_text_color = "#cccccc"
p.yaxis.major_label_text_color = "#cccccc"
p.xaxis.axis_line_color = "#333333"
p.yaxis.axis_line_color = "#333333"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "#222244"
p.title.text_color = "#eeeeee"
p.title.text_font_size = "16pt"
p.title.align = "center"
p.legend.label_text_color = "#ffffff"
p.legend.background_fill_alpha = 0.0
p.legend.location = "top_left"
p.legend.label_text_font_size = "9pt"

show(p)


C:\Users\mich0\AppData\Local\Temp\ipykernel_16168\416246582.py:9: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("complete.csv", on_bad_lines='skip')


In [33]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
from bokeh.palettes import Category20
from collections import Counter

# Load and clean data
data = pd.read_csv("complete.csv", on_bad_lines='skip')
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
df = data[data["country"] == "us"].dropna(subset=['datetime', 'shape'])
df['year'] = df['datetime'].dt.year

# Group by year and shape
shape_counts = df.groupby(['year', 'shape']).size().unstack(fill_value=0)

# Most common shapes to display
top_shapes = shape_counts.sum().sort_values(ascending=False).head(6).index
shape_counts = shape_counts[top_shapes]
shape_counts_sorted = shape_counts.sort_index()

# Emoji mappings for top shapes
emoji_map = {
    'light': '💡',
    'triangle': '🔺',
    'circle': '⭕',
    'fireball': '🔥',
    'unknown': '❓',
    'other': '🛸'
}
# Replace any missing with placeholder
for shape in top_shapes:
    emoji_map.setdefault(shape, '🛸')

# Cumulative data for stacking
source_data = {'year': shape_counts_sorted.index.tolist()}
colors = Category20[20][:len(top_shapes)]
for shape in top_shapes:
    source_data[shape] = shape_counts_sorted[shape].values

source = ColumnDataSource(data=source_data)

# === Media Events ===
alien_media = {
    1951: "The Day the Earth Stood Still",
    1977: "Close Encounters",
    1979: "Alien",
    1982: "E.T.",
    1986: "Aliens",
    1993: "The X-Files (TV)",
    1996: "Independence Day",
    1998: "X-Files: The Movie",
    2002: "Signs",
    2009: "District 9",
    2014: "Edge of Tomorrow"
}

# Media marker setup: place dot on total sightings for that year
media_years, media_titles, media_counts = [], [], []
total_per_year = shape_counts_sorted.sum(axis=1)
for year, title in alien_media.items():
    if year in total_per_year.index:
        media_years.append(year)
        media_titles.append(title)
        media_counts.append(total_per_year[year])

media_source = ColumnDataSource(data=dict(
    year=media_years,
    count=media_counts,
    title=media_titles
))

# Create base figure
p = figure(
    width=1000, height=500,
    x_range=(shape_counts_sorted.index.min(), shape_counts_sorted.index.max()),
    y_range=(0, total_per_year.max() * 1.15),
    background_fill_color="#0f0f1f",
    title="U.S. UFO Shape Distribution Over Time with Major Media Releases",
    tools="", toolbar_location=None
)

# === Stack Area Plot ===
p.varea_stack(stackers=list(top_shapes), x='year',
              color=colors, legend_label=list(top_shapes), source=source)

# === Media dots ===
media_renderer = p.circle(x='year', y='count', size=10, color="#ff3366",
                          alpha=0.9, source=media_source, legend_label="Media Event")

# === Tooltip for media events ===
p.add_tools(HoverTool(
    renderers=[media_renderer],
    tooltips=[("🎬 Media", "@title"), ("📅 Year", "@year"), ("📈 Sightings", "@count")],
    mode='mouse'
))

# === Emoji Labels (center of each shape band in last year) ===
emoji_labels = []
final_year = shape_counts_sorted.index.max()
final_counts = shape_counts_sorted.loc[final_year]
cumulative = 0

for shape in top_shapes:
    mid_point = cumulative + final_counts[shape] / 2
    emoji_labels.append((final_year, mid_point, emoji_map.get(shape, '🛸')))
    cumulative += final_counts[shape]

emoji_source = ColumnDataSource(data=dict(
    x=[e[0] for e in emoji_labels],
    y=[e[1] for e in emoji_labels],
    emoji=[e[2] for e in emoji_labels]
))

p.add_layout(LabelSet(x='x', y='y', text='emoji', source=emoji_source,
                      text_font_size="18pt", text_align="center", text_baseline="middle"))

# === Style ===
p.xaxis.axis_label = "Year"
p.yaxis.axis_label = "Sightings"
p.xaxis.major_label_text_color = "#cccccc"
p.yaxis.major_label_text_color = "#cccccc"
p.xaxis.axis_line_color = "#333333"
p.yaxis.axis_line_color = "#333333"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "#222244"
p.title.text_color = "#eeeeee"
p.title.text_font_size = "16pt"
p.title.align = "center"
p.legend.label_text_color = "#ffffff"
p.legend.background_fill_alpha = 0.0
p.legend.location = "top_left"
p.legend.label_text_font_size = "9pt"

# Show plot
show(p)


C:\Users\mich0\AppData\Local\Temp\ipykernel_16168\3716458546.py:8: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("complete.csv", on_bad_lines='skip')


In [38]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
from bokeh.palettes import Category20
from collections import Counter

# ------------------------------------------------------------
# 1. Load & clean
# ------------------------------------------------------------
data = pd.read_csv("complete.csv", on_bad_lines='skip')
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
df   = data[(data['country'] == 'us') & data['shape'].notna()].copy()
df['year'] = df['datetime'].dt.year

# ------------------------------------------------------------
# 2. Counts per (year, shape)  – keep 6 most‑reported shapes
# ------------------------------------------------------------
shape_counts = (
    df.groupby(['year', 'shape']).size()
      .unstack(fill_value=0)
      .sort_index()
)
top_shapes = shape_counts.sum().sort_values(ascending=False).head(6).index
shape_counts = shape_counts[top_shapes]

# ------------------------------------------------------------
# 3. Data sources for stacking
# ------------------------------------------------------------
stack_source = ColumnDataSource(
    dict(year=shape_counts.index.tolist(),
         **{sh: shape_counts[sh].values for sh in top_shapes})
)
colors = Category20[20][:len(top_shapes)]

# ------------------------------------------------------------
# 4. Emoji map + positions for icon labels
# ------------------------------------------------------------
emoji_map = {'light':'💡', 'triangle':'🔺', 'circle':'⭕',
             'fireball':'🔥', 'unknown':'❓', 'other':'🛸'}

peak_years = shape_counts.idxmax()
label_x, label_y, label_txt, label_size = [], [], [], []
max_ht = shape_counts.values.max()

for sh in top_shapes:
    yr = int(peak_years[sh])
    layer_ht = shape_counts.loc[yr, sh]
    baseline  = shape_counts.loc[yr, :].cumsum() - layer_ht
    mid_y = baseline[sh] + layer_ht / 2

    label_x.append(yr)
    label_y.append(mid_y)
    label_txt.append(emoji_map.get(sh, '🛸'))
    label_size.append(f"{max(10, min(20, int(layer_ht/max_ht*30)))}pt")

label_source = ColumnDataSource(dict(
    x=label_x, y=label_y, emoji=label_txt, size=label_size
))

# ------------------------------------------------------------
# 5. Media‑event dots
# ------------------------------------------------------------
alien_media = {
    1951:"The Day the Earth Stood Still", 1977:"Close Encounters",
    1979:"Alien", 1982:"E.T.", 1986:"Aliens", 1993:"The X‑Files (TV)",
    1996:"Independence Day", 1998:"X‑Files: The Movie",
    2002:"Signs", 2009:"District 9", 2014:"Edge of Tomorrow"
}
totals = shape_counts.sum(axis=1)
media_source = ColumnDataSource(dict(
    year  =[y for y in alien_media if y in totals],
    count =[totals[y] for y in alien_media if y in totals],
    title =[alien_media[y] for y in alien_media if y in totals]
))

# ------------------------------------------------------------
# 6. Figure
# ------------------------------------------------------------
p = figure(
    width=1000, height=500,
    x_range=(shape_counts.index.min(), shape_counts.index.max()),
    y_range=(0, totals.max()*1.1),
    background_fill_color="#0f0f1f",
    title="U.S. UFO Shape Distribution Over Time with Major Media Releases",
    tools="", toolbar_location=None       # lock pan/zoom
)

# ---------- stacked areas with emoji‑enhanced legend ----------
legend_labels = [f"{emoji_map.get(sh, '🛸')} {sh}" for sh in top_shapes]
p.varea_stack(stackers=list(top_shapes), x='year', source=stack_source,
              color=colors, legend_label=legend_labels)

# ---------- media dots ----------
media_renderer = p.circle('year', 'count', size=10, color="#ff3366",
                          alpha=0.9, source=media_source,
                          legend_label="Media Event")
p.add_tools(HoverTool(renderers=[media_renderer],
                      tooltips=[("🎬", "@title"),
                                ("Year", "@year"),
                                ("Sightings", "@count")]))

# ---------- emoji labels inside bands ----------
p.add_layout(LabelSet(x='x', y='y', text='emoji', source=label_source,
                      text_font_size={'field':'size'},
                      text_align="center", text_baseline="middle"))

# ---------- styling ----------
p.xaxis.axis_label, p.yaxis.axis_label = "Year", "Sightings"
for ax in (p.xaxis, p.yaxis):
    ax.major_label_text_color, ax.axis_line_color = "#ccc", "#333"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "#222244"
p.title.text_color, p.title.text_font_size = "#eee", "16pt"
p.legend.location = "top_left"
p.legend.background_fill_alpha = 0.0
p.legend.label_text_color = "#fff"
p.legend.label_text_font_size = "9pt"

show(p)


C:\Users\mich0\AppData\Local\Temp\ipykernel_16168\1772442291.py:10: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("complete.csv", on_bad_lines='skip')
